### Done


### ToDo


# CartPole with Policy Gradient

In [6]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',

    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

<IPython.core.display.Javascript object>

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Standard libraries

# 3rd party libraries
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import gym

# Custom libraries
import CartPole_config as config
import utils
import networks

In [8]:
# Settings and parameters
experiment_name = None
experiment_name = 'CartPole_PG'

batch_size = 5
LEARNING_RATE = 1e-2


In [9]:
## Derrived settings
experiment_name = experiment_name or utils.time_str()
logdir = './logdir/'+config.env_name+'/A3C/' + experiment_name
print('logdir: ', logdir)


logdir:  ./logdir/CartPole-v1/A3C/CartPole_PG


In [13]:
%%time
from tensorflow.contrib.keras.api.keras.layers import Dense, Input
from tensorflow.contrib.keras.api.keras.models import Model


Wall time: 0 ns


In [16]:

## Create model
tf.reset_default_graph()

class PolicyGradient:
    
    def __init__(self, render=False):
        self.env = gym.make(config.env_name)
        self.render = render
        self.should_stop = False
        
        self.obsPH = tf.placeholder(tf.float32, shape=[None]+[config.num_state], name='obsPlaceholder')
        self.actionPH = tf.placeholder(tf.int32, shape=[None, 1], name='actionPlaceholder')
        self.advantagePH = tf.placeholder(tf.float32, shape=[None, 1], 
                            name='advantagePlaceholder')

        self.model = self._build_model()
        self.graph = self._build_graph(LEARNING_RATE)

        self.saver = tf.train.Saver(max_to_keep=5)
        
    def load_model(self, path):
        ckpt = tf.train.get_checkpoint_state(path)
        self.saver.restore(self.sess, ckpt.model_checkpoint_path)
    
    def save_model(self, path):
        self.saver.save(self.sess, path)

    def _build_model(self):
        input_layer = Input(tensor=self.obsPH)
        model_layers = networks.build_dense(input_layer, config.layers, name_stem='dense_')
        model = Model(inputs=input_layer, outputs=model_layers)
        return model

    def _build_graph(self, learning_rate):
        class PGGraph: pass
        graph = PGGraph
        
        with tf.variable_scope('actor'):
            action_hot = tf.one_hot(self.actionPH, config.num_action)
            graph.action_probs = Dense(config.num_action, 
                                   activation='softmax')(self.model.output)
            graph.action_prob = tf.reduce_sum(graph.action_probs * action_hot,
                                    axis=1, keep_dims=True)
            log_prob = tf.log(graph.action_prob)
            graph.loss_total = - tf.reduce_mean(log_prob * self.advantagePH)
            
            optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
            
            gradients, variables = optimizer.compute_gradients(graph.loss_total)
            
            # Summary of gradients and variables!
            
            clipped_gradients = [(tf.clip_by_value(grad, -1., 1.), var
                                 for grad, var in zip(gradients, variables))]
            
            graph.train_op = optimizer.apply_gradients(clipped_gradients)
            
            
    def stop(self):
        self.should_stop = True

    def get_action(self, obs):
        """ Takes a single obs, and returns a single action"""
        obs = [obs]
        [p] = self.sess.run(
                self.graph.action_probs,
                feed_dict={self.brain.obsPH : obs})
        a = np.random.choice(config.num_action, p=p)
        return a
    
    def run(self):
        self.sess = tf.Session()
        obs_prev = self.env.reset()
        try:
            while self.should_stop is False:
                action = get_action(obs_prev)
                obs_new, reward, done, _ = self.env.step(action)
                if self.render: self.env.render()
                
                # add experience to memory
                
                obs_prev = obs_new
                    
                if done:
                    
                    # stack experience
                    
                    # compute discounted rewards
                    
                    # normalize discounted rewards
                
                
        except KeyboardInterrupt:
            print('KeyboardInterrupt')

agent = PolicyGradient()
print('done')

done
